In [1]:
import pandas as pd
import numpy as np
from utils.io import gdf_merger

In [2]:
df1 = pd.DataFrame({'ID': ['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', np.nan, ],
                   'X': [152890, np.nan, 152821, 152815, 152875, 152885, 152810, 152861, 152800],
                   'Y': [122596, np.nan, 122512, 122565, 122575, 122581, 122556, 122523, 122546],
                   'Name': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'i'],
                   'Value': [1, 2, 3, 4, np.nan, 5, np.nan, 7, 8]})
df2 = pd.DataFrame({'ID': ['id1', 'id2', 'id3', 'id4', 'id5', 'id6', 'id7', 'id8', np.nan, np.nan],
                   'X': [152890, 152885, 152821, 152865, 152875, np.nan, 152845, 152861, 152800, 152900],
                   'Y': [122596, 122500, 122556, 122565, 122575, np.nan, 122556, 122523, 122546, 122600],
                   'Name': ['A', 'B', 'D', 'C', 'E', 'F', 'G', 'h', 'i', 'k'],
                   'Value': [1, 3, 2, 4, np.nan, np.nan, 6, 7, 8, 9]})

In [3]:
df1

,ID,X,Y,Name,Value
0,id1,152890.0,122596.0,A,1.0
1,id2,NaN,NaN,B,2.0
2,id3,152821.0,122512.0,C,3.0
3,id4,152815.0,122565.0,D,4.0
4,id5,152875.0,122575.0,E,NaN
5,id6,152885.0,122581.0,F,5.0
6,id7,152810.0,122556.0,G,NaN
7,id8,152861.0,122523.0,H,7.0
8,NaN,152800.0,122546.0,i,8.0


In [4]:
df2

,ID,X,Y,Name,Value
0,id1,152890.0,122596.0,A,1.0
1,id2,152885.0,122500.0,B,3.0
2,id3,152821.0,122556.0,D,2.0
3,id4,152865.0,122565.0,C,4.0
4,id5,152875.0,122575.0,E,NaN
5,id6,NaN,NaN,F,NaN
6,id7,152845.0,122556.0,G,6.0
7,id8,152861.0,122523.0,h,7.0
8,NaN,152800.0,122546.0,i,8.0
9,NaN,152900.0,122600.0,k,9.0


In [5]:
gdf_merger(df1, df2, col='ID')[0]


Columns dropped :['X_x', 'Y_x', 'Name_x', 'Value_x', 'X_y', 'Y_y', 'Name_y', 'Value_y']

1 NaN lines dropped


/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pandas/core/frame.py:4305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,ID,X,Y,Name,Value
0,id1,152890.0,122596.0,A,1.0
1,id2,152885.0,122500.0,B,NaN
2,id3,152821.0,NaN,NaN,NaN
3,id4,NaN,122565.0,NaN,4.0
4,id5,152875.0,122575.0,E,NaN
5,id6,152885.0,122581.0,F,5.0
6,id7,NaN,122556.0,G,6.0
7,id8,152861.0,122523.0,H,7.0
8,NaN,152800.0,122546.0,i,8.0


In [6]:
mdf = pd.merge(df1,df2, on='ID')

In [7]:
mdf

,ID,X_x,Y_x,Name_x,Value_x,X_y,Y_y,Name_y,Value_y
0,id1,152890.0,122596.0,A,1.0,152890.0,122596.0,A,1.0
1,id2,NaN,NaN,B,2.0,152885.0,122500.0,B,3.0
2,id3,152821.0,122512.0,C,3.0,152821.0,122556.0,D,2.0
3,id4,152815.0,122565.0,D,4.0,152865.0,122565.0,C,4.0
4,id5,152875.0,122575.0,E,NaN,152875.0,122575.0,E,NaN
5,id6,152885.0,122581.0,F,5.0,NaN,NaN,F,NaN
6,id7,152810.0,122556.0,G,NaN,152845.0,122556.0,G,6.0
7,id8,152861.0,122523.0,H,7.0,152861.0,122523.0,h,7.0
8,NaN,152800.0,122546.0,i,8.0,152800.0,122546.0,i,8.0
9,NaN,152800.0,122546.0,i,8.0,152900.0,122600.0,k,9.0


In [8]:
mdf['conflict_Name'] = mdf['Name_x'].apply(lambda x: x.lower())!=mdf['Name_y'].apply(lambda x: x.lower())

In [9]:
mdf['conflict_Value'] = (mdf['Value_x']!=mdf['Value_y']) & ~(np.isnan(mdf['Value_x']) | np.isnan(mdf['Value_y']))

In [10]:
dist_max = 2.

In [11]:
mdf['distinct_objects'] = ((mdf['X_x']-mdf['X_y'])**2+(mdf['Y_x']-mdf['Y_y'])**2) > dist_max**2

In [12]:
mdf

,ID,X_x,Y_x,Name_x,Value_x,X_y,Y_y,Name_y,Value_y,conflict_Name,conflict_Value,distinct_objects
0,id1,152890.0,122596.0,A,1.0,152890.0,122596.0,A,1.0,False,False,False
1,id2,NaN,NaN,B,2.0,152885.0,122500.0,B,3.0,False,True,False
2,id3,152821.0,122512.0,C,3.0,152821.0,122556.0,D,2.0,True,True,True
3,id4,152815.0,122565.0,D,4.0,152865.0,122565.0,C,4.0,True,False,True
4,id5,152875.0,122575.0,E,NaN,152875.0,122575.0,E,NaN,False,False,False
5,id6,152885.0,122581.0,F,5.0,NaN,NaN,F,NaN,False,False,False
6,id7,152810.0,122556.0,G,NaN,152845.0,122556.0,G,6.0,False,False,True
7,id8,152861.0,122523.0,H,7.0,152861.0,122523.0,h,7.0,False,False,False
8,NaN,152800.0,122546.0,i,8.0,152800.0,122546.0,i,8.0,False,False,False
9,NaN,152800.0,122546.0,i,8.0,152900.0,122600.0,k,9.0,True,True,True


In [13]:
mdf['conflict'] = ((mdf['conflict_Name']==True) | (mdf['conflict_Value']==True)) & (mdf['distinct_objects']==False)

In [14]:
mdf

,ID,X_x,Y_x,Name_x,Value_x,X_y,Y_y,Name_y,Value_y,conflict_Name,conflict_Value,distinct_objects,conflict
0,id1,152890.0,122596.0,A,1.0,152890.0,122596.0,A,1.0,False,False,False,False
1,id2,NaN,NaN,B,2.0,152885.0,122500.0,B,3.0,False,True,False,True
2,id3,152821.0,122512.0,C,3.0,152821.0,122556.0,D,2.0,True,True,True,False
3,id4,152815.0,122565.0,D,4.0,152865.0,122565.0,C,4.0,True,False,True,False
4,id5,152875.0,122575.0,E,NaN,152875.0,122575.0,E,NaN,False,False,False,False
5,id6,152885.0,122581.0,F,5.0,NaN,NaN,F,NaN,False,False,False,False
6,id7,152810.0,122556.0,G,NaN,152845.0,122556.0,G,6.0,False,False,True,False
7,id8,152861.0,122523.0,H,7.0,152861.0,122523.0,h,7.0,False,False,False,False
8,NaN,152800.0,122546.0,i,8.0,152800.0,122546.0,i,8.0,False,False,False,False
9,NaN,152800.0,122546.0,i,8.0,152900.0,122600.0,k,9.0,True,True,True,False


In [15]:
distinct = mdf.query('distinct_objects==True')

In [16]:
mdf['X_y'] = mdf[['X_x', 'X_y', 'distinct_objects']].apply(lambda x: x[0] if x[2]==True else x[1], axis=1)

In [17]:
mdf['Y_y'] = mdf[['Y_x', 'Y_y', 'distinct_objects']].apply(lambda x: x[0] if x[2]==True else x[1], axis=1)

In [18]:
mdf['Value_y'] = mdf[['Value_x', 'Value_y', 'distinct_objects']].apply(lambda x: x[0] if x[2]==True else x[1], axis=1)

In [19]:
mdf['Name_y'] = mdf[['Name_x', 'Name_y', 'distinct_objects']].apply(lambda x: x[0] if x[2]==True else x[1], axis=1)

In [20]:
mdf

,ID,X_x,Y_x,Name_x,Value_x,X_y,Y_y,Name_y,Value_y,conflict_Name,conflict_Value,distinct_objects,conflict
0,id1,152890.0,122596.0,A,1.0,152890.0,122596.0,A,1.0,False,False,False,False
1,id2,NaN,NaN,B,2.0,152885.0,122500.0,B,3.0,False,True,False,True
2,id3,152821.0,122512.0,C,3.0,152821.0,122512.0,C,3.0,True,True,True,False
3,id4,152815.0,122565.0,D,4.0,152815.0,122565.0,D,4.0,True,False,True,False
4,id5,152875.0,122575.0,E,NaN,152875.0,122575.0,E,NaN,False,False,False,False
5,id6,152885.0,122581.0,F,5.0,NaN,NaN,F,NaN,False,False,False,False
6,id7,152810.0,122556.0,G,NaN,152810.0,122556.0,G,NaN,False,False,True,False
7,id8,152861.0,122523.0,H,7.0,152861.0,122523.0,h,7.0,False,False,False,False
8,NaN,152800.0,122546.0,i,8.0,152800.0,122546.0,i,8.0,False,False,False,False
9,NaN,152800.0,122546.0,i,8.0,152800.0,122546.0,i,8.0,True,True,True,False


In [21]:
distinct.loc[:,'X_x'] = distinct[['X_x', 'X_y', 'distinct_objects']].apply(lambda x: x[1] if x[2]==True else x[0], axis=1)

/home/su530201/.local/share/virtualenvs/GSDMA-tJb-HgI6/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [22]:
distinct.loc[:,'Y_x'] = distinct[['Y_x', 'Y_y', 'distinct_objects']].apply(lambda x: x[1] if x[2]==True else x[0], axis=1)

In [23]:
mdf['Value_y'] = mdf[['Value_x', 'Value_y', 'distinct_objects']].apply(lambda x: x[0] if x[2]==True else x[1], axis=1)

In [24]:
mdf['Name_y'] = mdf[['Name_x', 'Name_y', 'distinct_objects']].apply(lambda x: x[0] if x[2]==True else x[1], axis=1)

In [25]:
distinct.loc[:,'Value_x'] = distinct[['Value_x', 'Value_y', 'distinct_objects']].apply(lambda x: x[1] if x[2]==True else x[0], axis=1)

In [26]:
distinct.loc[:,'Name_x'] = distinct[['Name_x', 'Name_y', 'distinct_objects']].apply(lambda x: x[1] if x[2]==True else x[0], axis=1)

In [27]:
distinct

,ID,X_x,Y_x,Name_x,Value_x,X_y,Y_y,Name_y,Value_y,conflict_Name,conflict_Value,distinct_objects,conflict
2,id3,152821.0,122556.0,D,2.0,152821.0,122556.0,D,2.0,True,True,True,False
3,id4,152865.0,122565.0,C,4.0,152865.0,122565.0,C,4.0,True,False,True,False
6,id7,152845.0,122556.0,G,6.0,152845.0,122556.0,G,6.0,False,False,True,False
9,NaN,152900.0,122600.0,k,9.0,152900.0,122600.0,k,9.0,True,True,True,False


In [28]:
mdf_2 = mdf.append(distinct)

Dataframe des conflits 

In [29]:
conflicts_to_resolve = mdf_2.query('conflict==True')

In [30]:
conflicts_to_resolve

,ID,X_x,Y_x,Name_x,Value_x,X_y,Y_y,Name_y,Value_y,conflict_Name,conflict_Value,distinct_objects,conflict
1,id2,NaN,NaN,B,2.0,152885.0,122500.0,B,3.0,False,True,False,True


Dataframe des row sans conflits

In [31]:
mdf_2 = mdf_2.query('conflict==False')

In [32]:
mdf_2['Value'] = mdf_2[['Value_x', 'Value_y']].apply(lambda x: x[1] if pd.isnull(x[0]) else x[0], axis=1)

In [33]:
mdf_2['Name'] = mdf_2[['Name_x', 'Name_y']].apply(lambda x: x[1] if pd.isnull(x[0]) else x[0], axis=1)

In [34]:
mdf_2['X'] =  mdf_2[['X_x', 'X_y']].apply(lambda x: x[1] if pd.isnull(x[0]) else x[0], axis=1)

In [35]:
mdf_2['Y'] =  mdf_2[['Y_x', 'Y_y']].apply(lambda x: x[1] if pd.isnull(x[0]) else x[0], axis=1)

In [36]:
mdf_2 = mdf_2[['ID', 'X', 'Y', 'Name', 'Value']]

In [37]:
mdf_2.reset_index(drop=True, inplace=True)

In [38]:
mdf_2

,ID,X,Y,Name,Value
0,id1,152890.0,122596.0,A,1.0
1,id3,152821.0,122512.0,C,3.0
2,id4,152815.0,122565.0,D,4.0
3,id5,152875.0,122575.0,E,NaN
4,id6,152885.0,122581.0,F,5.0
5,id7,152810.0,122556.0,G,NaN
6,id8,152861.0,122523.0,H,7.0
7,NaN,152800.0,122546.0,i,8.0
8,NaN,152800.0,122546.0,i,8.0
9,id3,152821.0,122556.0,D,2.0


In [39]:
# reste à tester si deux objets aved des id différents ne sont pas en réalité un seul et même objet -> très compliqué à faire sans verification sur le terrain